# Executing Brainfuck at Compile Time with C++ Templates

13.06.2016, C++ User Group Hannover


Jacek Galowicz ([jacek@galowicz.de](mailto:jacek@galowicz.de))

## Content

I will show you how to implement a brainfuck interpreter, which runs your own brainfuck programs in the C++ compiler, at compile time.

### Talk sections

- Short intro to Brainfuck
- Compile Time Type Lists
- The Turing Tape
- The Turing Machine State
- Interpreting BF Commands
- Reaching today's climax of complicatedness: Loops
- Live Demo
- Summary


# Brainfuck

It's an esoteric programming language, and was created for fun in 1993 by Urban Müller. [Wikipedia Link](https://en.wikipedia.org/wiki/Brainfuck)

- Brainfuck programs are composed of only 8 operators, and assume to operate on a Brainfuck Machine.
- A Brainfuck machine looks like a Turing machine: It has a cursor, which sits on a tape, which consists of infinitely many memory cells.

|Operator|Meaning|
|:------:|-------|
|`+`|Increment the value of the memory cell at data cursor position|
|`-`|Increment the value of the memory cell at data cursor position|
|`<`|Move the data cursor one cell further to the left|
|`>`|Move the data cursor one cell further to the right|
|`.`|Print the value at data cursor position|
|`,`|Read a value, and assign it to the memory cell at data cursor position|
|`[`|Beginning of a loop. Execute it, if the data cursor value is not 0. Skip the whole loop, if it is 0.|
|`]`|End of a loop. Move program cursor to beginning of the loop.|

Any other character in a brainfuck program is ignored (Spaces can be used for nicer to read indentation etc.)

# Brainfuck Examples





## Simple Print Loop

The following program reads a value from the user, then prints and decrements it in a loop:

```
,[.-]
```

Output: (Assuming the user enters the character `z`)
```
zyxwvutsrqponmlkjihgfedcba`_^]\[ZYXWVUTSRQPONMLKJIHGFEDCBA@?>=<;:9876543210/.-,+*)('&%$#"!
```

## Hello World

The following program reads a value from the user, then prints and decrements it in a loop:

```
++++++++++[>+++++++>++++++++++>+++>+<<<<-]>++.>+.+++++++..+++.>++.<<+++++++++++++++.>.+++.------.--------.>+.>.+++.
```

Output: (Assuming the user enters the character `z`)
```
Hello World!
```

# Compile Time Typelists

The nested typelist



``` c++
struct null_t {};

template <typename T, typename U>
struct tl
{
    using head = T;
    using tail = U;
};

using my_list = tl<Type1, 
                   tl<Type2, 
                      tl<Type3, 
                         null_t
                         >
                      > 
                   >;
```

# Translating from C Strings to compile time lists



## Required: String --> Typelist

We need to represent user input as a list, in order to work with it:

``` c++
template <char val> struct char_t { 
    static const constexpr char value {val};
};
```

A type list, which represents the string "abc":
``` c++
using abc_list = tl<char_t<'a'>, tl<char_t<'b'>, tl<char_t<'c'>, null_t> > >;
```

## String --> String Provider

C-Style ```char*``` strings are no valid template parameters. 

We need to represent them as a type:

``` c++
struct abc_string_provider {
   static constexpr const char * str() {
       return "abc";
   }
};

using my_abc_string = string_list_t<abc_string_provider>;
```

The type ```my_abc_string``` is a valid template type parameter.

## String Provider --> Nested Typelist

 

``` c++
template <class Str, size_t Pos, char C>
struct string_list;

template <class Str, size_t Pos, char C>
struct string_list {
    using next_piece = typename string_list<
                            Str,
                            Pos + 1,
                            Str::str()[Pos + 1]
                        >::type;
    using type = tl<char_t<C>, next_piece>;
};

template <class Str, size_t Pos>
struct string_list<Str, Pos, '\0'> {
    using type = null_t;
};

template <class Str>
using string_list_t = typename string_list<Str, 0, Str::str()[0]>::type;
```

# The Turing Tape

We need a structure which models the cursor which points to memory cells on a tape of infinite length.

So let's first create a type, which can represent such a tape:

``` c++
template <class LList, class Cursor, class RList>
struct tape;
```

``` c++
template <class LHead, class LTail, 
          class Cursor, 
          class RHead, class RTail>
struct tape<
           tl<LHead, LTail>,
           Cursor,
           tl<RHead, RTail>
       > {
       
    using get = Cursor;

    template <class T>
    using set = tape<
                    tl<LHead, LTail>, 
                    T, 
                    tl<RHead, RTail>>;

    using move_left  = tape<
                           LTail, 
                           LHead, 
                           tl<Cursor, tl<RHead, RTail>>>;
    using move_right = tape<
                           tl<Cursor, tl<LHead, LTail>>, 
                           RHead, 
                           RTail>;
};
```

## Some Helpers, for readability

``` c++
template <class Tape>
using get_t = typename Tape::get;


template <class Tape, class T>
using set_t = typename Tape::template set<T>;


template <class Tape>
using move_left_t  = typename Tape::move_left;


template <class Tape>
using move_right_t = typename Tape::move_right;


template <class T>
using make_t = tape<null_t, T, null_t>;
```

## Some Example Use

``` c++
using a_tape       = make_t<char_t<'a'>>;

using shifted_left = move_left_t<a_tape>;

using set_to_b     = set_t<shifted_left, char_t<'b'>>;

// Or in just one line:
using ab_tape = set_t<move_left_t<make_t<char_t<'a'> > >, char_t<'b'> >;
```

# The Brainfuck Machine

It's basically a turing tape:

``` c++
template <typename Tape>
struct bf_machine;
```

... with some little extras:

``` c++
template <typename Tape>
struct machine {
    using move_left  = machine<null_to_0_t<tt_move_left_t< Tape> > >;
    
    using move_right = machine<null_to_0_t<tt_move_right_t<Tape> > >;

    using get = get_t<Tape>;
    
    template <char value>
    using set = machine<set_t<Tape, char_t<value> > >;

    static const constexpr char value {get::value};

    using increment = set<value + 1>;
    using decrement = set<value - 1>;
};
```

``` c++
// Brainfuck '<'
template <typename Machine>
using move_left_t = typename Machine::move_left;

// Brainfuck '>'
template <typename Machine>
using move_right_t = typename Machine::move_right;

// Brainfuck '.'
template <typename Machine>
using get_t = typename Machine::get;

// Brainfuck ','
template <typename Machine, char val>
using set_t = typename Machine::template set<val>;

// Brainfuck '+'
template <typename Machine>
using increment_t = typename Machine::increment;

// Brainfuck '-'
template <typename Machine>
using decrement_t = typename Machine::decrement;

using make_t = machine<tape_make_t<char_t<0>>>;
```

# BFM + I/O

We have a brainfuck machine, which can be used to manually execute BF operations.

In order to do more with it, we can wrap it into another type:

``` c++
template <
    class BFM, 
    class Inlist, 
    class OutList>
struct io_bfm;
```

The next step is a function, which takes an `IOBFM` state and a brainfuck command, and returns a new `IOBFM` state.

\begin{matrix}
   f: & (state_{old}, char_{input}) & \to & state_{new}\\
   & & & \\
   f: & (state_{old}, +) & \to & state_{incremented}\\
   f: & (state_{old}, >) & \to & state_{moved\ right}\\
      & & \dots & 
\end{matrix}

in C++:

``` c++
template <class IOBFM, char InputChar>
struct interpret_step;

template <class IOBFM, char InputChar>
using interpret_step_t = typename interpret_step<IOBFM, InputChar>::type;
```

``` c++
// '+' command: Increment cursor value

template <class BFM, class InList, class OutList>
struct interpret_step<io_bfm<BFM, InList, OutList>, '+'> {
    using type = io_bfm<increment_t<BFM>,
                        InList, 
                        OutList>;
};
```

``` c++
// '>' command: Move cursor to the right

template <class BFM, class InList, class OutList>
struct interpret_step<io_bfm<BFM, InList, OutList>, '>'> {
    using type = io_bfm<move_right_t<BFM>, 
                        InList, 
                        OutList>;
};
```

``` c++
// '.' command: Output cursor value

template <class BFM, class InList, class OutList>
struct interpret_step<io_bfm<BFM, InList, OutList>, '.'> {
    using type = io_bfm<BFM, 
                       InList, 
                       append_t<OutList, get_t<BFM>>>;
};
```

``` c++
// ',' command: Assign input value to cursor

template <class BFM, class InList, class OutList>
struct interpret_step<io_bfm<BFM, InList, OutList>, ','> {
    using type = io_bfm<set_t<BFM, head_t<InList>::value>, 
                        tail_t<InList>, 
                        OutList>;
};
```

# BFM + I/O + BF Program

We can now call ```interpret_step``` with an ```io_bfm``` state, and get the next state.

We can do this in a loop, which feeds a Brainfuck Program (stored in another list) to this function, character by character, until the program runs empty.


\begin{matrix}
       & f: & ( & state_{old},   & (, + + .) & ) \\
   \to & f: & ( & state_{new1},  & (+ + .)   & )  \\
   \to & f: & ( & state_{new2},  & (+ .)     & )  \\
   \to & f: & ( & state_{new3},  & (.)       & )  \\
   \to &    &   & state_{final}  &
\end{matrix}

The loop:

``` c++
template <class IOBFM, class ProgList>
struct run_tm;

template <class IOBFM, class ProgList>
using run_tm_t = typename run_tm<IOBFM, ProgList>::type;
```

``` c++
template <class IOBFM, char Command, class RestProg>
struct run_tm<IOBFM, tl<char_t<Command>, RestProg>> {
    using type = typename run_tm<
                      interpret_step_t<IOBFM, Command>, 
                      RestProg
                 >::type;
};

// Recursion end case:
template <class IOBFM>
struct run_tm<IOBFM, null_t> {
    using type = IOBFM;
};
```

We are covering `+`, `-`, `<`, `>` now.

But that's not all, there is still something missing!

# Loops: `[` `]`

Loops are pretty complicated, because we have to search for the next closing bracket `']'` character, in order to know what belongs to the loop body.

Loops can be nested: `[.........[...[...]......]...]`

Solution: Count bracket pairs.

Count **up** on opening brackets `'['`, and **down** on closing brackets `']'`.

```
......[......[......[.....]....].....]......
      1      2      3     2    1     X
```

We need a function, which takes the part of a BF program which starts with a loop, and which returns the loop body and the rest of the program:
    
\begin{equation*}
   f: "[xxxxx]yyyyyy" \to ("xxxxx", "yyyyyy")
\end{equation*}

``` c++
template <class InList, class OutList, size_t Counter>
struct find_brace;

template <class InList>
using find_brace_t = find_brace<InList, null_t, 1>;
```

``` c++
// Match: non-bracket characters

template <char C, class InList, class OutList, size_t N>
struct find_brace<tl<char_t<C>, InList>, 
                  OutList, 
                  N>
    // Add character to outlist
    : public find_brace<InList, 
                        append_t<OutList, char_t<C>>,
                        N>
{};
```

``` c++
// Match: counter is != 1, next character is '['

template <class InList, class OutList, size_t N>
struct find_brace<tl<char_t<'['>, InList>, 
                  OutList, 
                  N>
    // Add character to outlist, increment counter
    : public find_brace<InList, 
                        append_t<OutList, char_t<'['>>, 
                        N + 1>
{};
```

``` c++
// Match: counter is != 1, next character is ']'

template <class InList, class OutList, size_t N>
struct find_brace<tl<char_t<']'>, InList>, 
                  OutList, 
                  N>
    // Add character to outlist, decrement counter
    : public find_brace<InList, 
                       append_t<OutList, char_t<']'>>, 
                       N - 1>
{};
```

``` c++
// Match: counter is 1, next character is ']'
// This is the final closing bracket.

template <class InList, class OutList>
struct find_brace<tl<char_t<']'>, InList>, 
                  OutList, 
                  1> {
    // We're done. The user can now choose...

    // ...the inner-loop program part
    using loop_block = OutList;

    // ...the rest of the program after the loop
    using rest_prog   = InList;
};
```

What a monstrum.

How to integrate that into our `run_tm` function in order to enable it for loop execution?

`run_tm` did formerly just take characters from the program list, and fed them into `interpret_step_t`.

We add a special case to it: If the next program character is `'['`, use our loop counting monster:

``` c++
// Matches: '[', Next BF command is the beginning of a loop
    
template <class IOBFM, class RestProg>
struct run_tm<IOBFM, tl<char_t<'['>, RestProg>> {

    static const constexpr bool loop_terminated {
                 get_t<typename IOBFM::state>::value == 0};

    using blocks = find_brace_t<RestProg>;

    // If the loop shall already terminate...
    using type = typename if_else_t<loop_terminated,
        // ...then run the rest of the program,
        //    which begins after the closing ']'
        run_tm<IOBFM, typename blocks::rest_prog>,

        // ...else, execute the BFM on the loop body,
        //    as if it was the whole program...
        run_tm<
            typename run_tm<IOBFM, 
                            typename blocks::loop_block
                      >::type,
            // ...and then confront it with the
            //    same loop again.
            tl<char_t<'['>, RestProg>>
    >::type;
};
```

# Demo

What happened during the demo:

Checkout the code from Github if you do not have it:

``` bash
git clone https://github.com/tfc/cpp_template_meta_brainfuck_interpreter.git
cd cpp_template_meta_brainfuck_interpreter
```

Output of the simple demo program from the beginning:

``` bash
Jacek.Galowicz ~/src/tmp_brainfuck $ g++ -o main main.cpp -O2 -std=c++11 -DINPUT_STR="\"z\"" -DPROGRAM_STR="\",[.-]\""
Jacek.Galowicz ~/src/tmp_brainfuck $ ./main
zyxwvutsrqponmlkjihgfedcba`_^]\[ZYXWVUTSRQPONMLKJIHGFEDCBA@?>=<;:9876543210/.-,+*)('&%$#"! 
```

Output of the "Hello World" program:

``` bash
Jacek.Galowicz ~/src/tmp_brainfuck $ g++ -o main main.cpp -O2 -std=c++11 -DPROGRAM_STR="\"++++++++++[>+++++++>++++++++++>+++>+<<<<-]>++.>+.+++++++..+++.>++.<<+++++++++++++++.>.+++.------.--------.>+.>.+++.\""
Jacek.Galowicz ~/src/tmp_brainfuck $ ./main
Hello World!
```

Proof that there is no real code executed other than the `puts(final string)` call, and the **static** string is directly loaded from the binaries text section:

``` bash
Jacek.Galowicz ~/src/tmp_brainfuck $ otool -tV main # Mac variant of objdump
main:
(__TEXT,__text) section
_main:
0000000100000f70	pushq	%rbp
0000000100000f71	movq	%rsp, %rbp
0000000100000f74	movq	0x85(%rip), %rdi        ## literal pool symbol address: __ZZN13tl_to_varlistIN2tl6null_tEJLc72ELc101ELc108ELc108ELc111ELc32ELc87ELc111ELc114ELc108ELc100ELc33ELc10ELc13EEE3strEvE6string
0000000100000f7b	callq	0x100000f84             ## symbol stub for: _puts
0000000100000f80	xorl	%eax, %eax
0000000100000f82	popq	%rbp
0000000100000f83	retq
```

# The Final Deal

``` c++
// The program input is provided by a string provider.
// The type list transformation article explains, how these work.
struct program_str { 
    static constexpr const char * str() { 
        // "Hello World" Brainfuck implementation from wikipedia
        return "++++++++++[>+++++++>++++++++++>+++>+<<<<-]>++.>+.+++++++." 
               ".+++.>++.<<+++++++++++++++.>.+++.------.--------.>+.>.+++.";
    } 
};

int main()
{
    // Transform the program string provider into a type list
    using prog       = string_list_t<program_str>;

    // Compose an initialized IOBFM from an empty BFM, and the empty input
    using BFM = bfm::io_bfm<bfm::make_t, null_t, null_t>;

    // The BFM's machine output is obtained by _running_ the IOBFM 
    // together with the brainfuck program in the run_tm function.
    using output = bfm::run_tm_t<BFM, prog>::output;

    // Generate a normal C string back from the output character typelist,
    // and finally print it.
    puts(tl_to_varlist<output>::str());

    return 0;
};
```

# Summary

- C++ TMP mechanisms are **turing complete**
- *Ugly syntax*, but nearly **1:1 translatable** between PF languages and TMP
- Great possibilities
- Brainfuck interpreter is a *fun project*
- I regard this meta program as a **demo vehicle** for interesting mechanisms/techniques.
    - There are some nice things i implemented at work with similar code, hence cannot show them here:
    - typesafe **python-style printf**: 
        - `print("Int: {} Long int: {} unsigned int: {} pointer: {}", 123, 123l, 123u, nullptr);`
    - Transforming symbol names into readable strings:
        - `TEST_CASE(data_structure_works_as_expected) { ... }`
        - print at runtime: `Test case: "data structure works as expexted": PASS`
    - `enable_if` with sets of types (stored in lists)
    - Automatic inheritance chains, defined by lists 
- Got the BF machine on **github**: [https://github.com/tfc/cpp_template_meta_brainfuck_interpreter](https://github.com/tfc/cpp_template_meta_brainfuck_interpreter)
- And covered it in multiple articles on my **blog**: [https://blog.galowicz.de](https://blog.galowicz.de)